In [42]:
import bw2data as bd
from collections import defaultdict
from tqdm import tqdm
from thefuzz import fuzz
import bw_processing as bwp
import numpy as np
import math
import stats_arrays as sa

In [2]:
bd.projects.set_current('GSA for archetypes')

In [3]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.8 cutoff
	swiss consumption 1.0

## Finding liquid fuel combustors

In [33]:
ei = bd.Database("ecoinvent 3.8 cutoff")

There are other combustion processes where fuels are measured in megajoules, these will be addressed later.

In [36]:
flows = ('market for diesel,', 'diesel,', 'petrol,', 'market for petrol,')
liquid_fuels = [x 
                for x in ei 
                if x['unit'] == 'kilogram'
                and((any(x['name'].startswith(flow) for flow in flows) or x['name'] == 'market for diesel'))
               ]
{x['name'] for x in liquid_fuels}

{'diesel, import from RoW',
 'diesel, low-sulfur, import from Europe',
 'diesel, low-sulfur, import from RoW',
 'market for diesel',
 'market for diesel, low-sulfur',
 'market for petrol, 15% ETBE additive by volume, with ethanol from biomass',
 'market for petrol, 4% ETBE additive by volume, with ethanol from biomass',
 'market for petrol, 5% ethanol by volume from biomass',
 'market for petrol, low-sulfur',
 'market for petrol, two-stroke blend',
 'market for petrol, unleaded',
 'petrol, low-sulfur, import from Europe',
 'petrol, unleaded, import from RoW'}

### Look into modelling specifics

In [18]:
a = liquid_fuels[0]
for production in a.production():
    pass

print(production['properties']['carbon content'])

{'amount': 0.865, 'unit': 'dimensionless', 'comment': 'carbon content on a dry matter basis (reserved; not for manual entry)'}


In [19]:
production

Exchange: 1.0 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None)>

In [12]:
consumer = list(a.consumers())[0]
consumer

Exchange: 0.0641344042785761 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RER, None)>

In [13]:
co2 = [x for x in bd.Database('biosphere3') if x['name'] == 'Carbon dioxide, fossil']
co2

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',))]

In [14]:
total_co2 = sum(exc['amount'] for exc in consumer.output.biosphere() if exc.input in co2)
total_co2

0.207834603648

In [20]:
consumer['amount'] / production['amount'] * production['properties']['carbon content']['amount'] * (12 + 16 * 2) / 12

0.20341295223688385

These numbers don't match because there is a second petrol input. Let's write a validation function.

In [38]:
def carbon_fuel_emissions_balanced(activity, fuels, co2):
    """Check that we can rescale carbon in fuel to CO2 emissions by checking their stoichiometry.
    
    Returns a ``bool`."""
    try:
        total_carbon = sum(
            # Carbon content amount is fraction of mass, unitless
            exc['amount'] * exc['properties']['carbon content']['amount'] 
            for exc in activity.technosphere() 
            if exc.input in fuels)
    except KeyError:
        return False
    conversion = 12 / (12 + 16 * 2)
    total_carbon_in_co2 = sum(
        exc['amount'] * conversion
        for exc in activity.biosphere()
        if exc.input in co2
    )
    return math.isclose(total_carbon, total_carbon_in_co2, rel_tol=1e-06, abs_tol=1e-3)

In [39]:
carbon_fuel_emissions_balanced(consumer.input, liquid_fuels, co2)

True

We also need an iterator that will return scaling factors sampled from the uncertainty. Note that we need to add all the carbon together to get the rescaling correct.

In [57]:
def get_samples_and_scaling_vector(activity, fuels, size=10, seed=None):
    """Draw ``size`` samples from technosphere exchanges for ``activity`` whose inputs are in ``fuels``.
    
    Returns:
        * Numpy indices array with shape ``(len(found_exchanges)),)``
        * Numpy flip array with shape ``(len(found_exchanges,))``
        * Numpy data array with shape ``(size, len(found_exchanges))``
        * Scaling vector with relative total carbon consumption and shape ``(size,)``.
    """
    exchanges = [exc for exc in activity.technosphere() if exc.input in fuels]
    static_total = sum(exc['amount'] * exc['properties']['carbon content']['amount'] for exc in exchanges)
    sample = sa.MCRandomNumberGenerator(
        sa.UncertaintyBase.from_dicts(*[exc.as_dict() for exc in exchanges]), 
        seed=seed
    ).generate(samples=size)
    indices = np.array([(exc.input.id, exc.output.id) for exc in exchanges], dtype=bwp.INDICES_DTYPE)
    carbon_fraction = np.array([exc['properties']['carbon content']['amount'] for exc in exchanges]).reshape((-1, 1))
    carbon_total_per_sample = (sample * carbon_fraction).sum(axis=0).ravel()
    flip = np.ones(indices.shape, dtype=bool)
    assert carbon_total_per_sample.shape == (size,)
    return indices, flip, sample, carbon_total_per_sample / static_total

In [58]:
get_samples_and_scaling_vector(consumer.output, liquid_fuels)

(array([(23091, 4569), ( 6596, 4569)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([ True,  True]),
 array([[0.06113076, 0.05726952, 0.06832648, 0.06287543, 0.10252436,
         0.0576822 , 0.07242146, 0.09426782, 0.05455941, 0.0759939 ],
        [0.00151065, 0.00099219, 0.00126345, 0.00119578, 0.00125752,
         0.00167235, 0.0014662 , 0.00105135, 0.0007399 , 0.00121496]]),
 array([0.95845283, 0.89144085, 1.06476959, 0.98032971, 1.58792794,
        0.90816197, 1.13052762, 1.45844315, 0.84611412, 1.18134398]))

In [56]:
def rescale_biosphere_exchanges_by_factors(activity, factors, flows):
    """Rescale biosphere exchanges with flows ``flows`` from ``activity`` by vector ``factors``.
    
    ``flows`` are biosphere flow objects, with e.g. all the CO2 flows, but also other flows such as metals, volatile organics, etc. 
    Only rescales flows in ``flows`` which are present in ``activity`` exchanges.
    
    Assumes the static values are balanced, i.e. won't calculate CO2 emissions from carbon in 
    fuels but just rescales given values.

    Returns: Numpy indices and data arrays with shape (number of exchanges found, len(factors)).
    Returns:
        * Numpy indices array with shape ``(len(found_exchanges)),)``
        * Numpy flip array with shape ``(len(found_exchanges,))``
        * Numpy data array with shape ``(len(found_exchanges), len(factors))``
    """
    indices, data = [], []
    assert isinstance(factors, np.ndarray) and len(factors.shape) == 1
    
    for exc in activity.biosphere():
        if exc.input in flows:
            indices.append((exc.input.id, exc.output.id))
            data.append(factors * exc['amount'])
            
    return np.array(indices, dtype=bwp.INDICES_DTYPE), np.zeros(len(indices), dtype=bool), np.vstack(data)

In [31]:
i, d = rescale_biosphere_exchanges_by_factors(consumer.output, np.linspace(0.8, 1.2, 10), co2)
i, d

(array([(118, 4569)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([[0.16626768, 0.17550478, 0.18474187, 0.19397896, 0.20321606,
         0.21245315, 0.22169024, 0.23092734, 0.24016443, 0.24940152]]))

In [32]:
d.shape

(1, 10)